# Introduction

## 1.1 Background
Jollibee is the largest fast food restaurant chain in the Philippines, while very well known around Southeast Asia and Middle East, there is only one Jollibee franchise in Toronto Canada.

## 1.2 Business Problem
Jollibee Foods Corporation (JFC) looks to expand its footprint in Toronto, Canada. As of July 2020, the only Jollibee franchise is in Scarborough and the company is looking for a similar neighborhood where they can open another franchise.

# Data

## 2.1 Data Sources
The neighbourhoods alongside their respective postal codes and boroughs were scraped from [Wikipedia](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M). Geographical coordinates for each neighbourhood were extracted from [here](https://cocl.us/Geospatial_data). I'll also be using the median household income data that is available [here](https://www03.cmhc-schl.gc.ca/hmip-pimh/en/TableMapChart/TableMatchingCriteria?GeographyType=MetropolitanMajorArea&GeographyId=2270&CategoryLevel1=Population%2C%20Households%20and%20Housing%20Stock&CategoryLevel2=Household%20Income&ColumnField=HouseholdIncomeRange&RowField=SurveyZone&SearchTags%5B0%5D.Key=Households&SearchTags%5B0%5D.Value=Number&SearchTags%5B1%5D.Key=Statistics&SearchTags%5B1%5D.Value=AverageAndMedian).

I will be utilizing the Foursquare API to obtain/explore which neighborhoods has the highest frequency of fast food chains.

## 